In [14]:
import numpy as np

from elastica.modules import BaseSystemCollection, Constraints, Forcing, CallBacks, Damping
from elastica.rod.cosserat_rod import CosseratRod
from elastica.dissipation import AnalyticalLinearDamper
from elastica.external_forces import GravityForces, MuscleTorques, EndpointForces
from elastica.interaction import AnisotropicFrictionalPlane
from elastica.boundary_conditions import OneEndFixedBC

from elastica.timestepper.symplectic_steppers import PositionVerlet
from elastica.timestepper import integrate

from elastica.callback_functions import CallBackBaseClass
from collections import defaultdict

from IPython.display import Video
from tqdm import tqdm

print("imports done")



class OctoArmSimulator(BaseSystemCollection, Constraints, Forcing, CallBacks, Damping):
    pass

octopus_sim = OctoArmSimulator()

n_elem = 50
start = np.array([0.0, 0.0, 0.0])
direction = np.array([0.0, 0.0, 1.0])
normal = np.array([0.0, 1.0, 0.0])
base_length = 0.35
base_radius = base_length * 0.011
base_area = np.pi * base_radius ** 2
density = 1000
nu = 2e-3
E = 1e6
poisson_ratio = 0.5
shear_modulus = E / (poisson_ratio + 1.0)

octo_arm = CosseratRod.straight_rod(
    n_elem,
    start,
    direction,
    normal,
    base_length,
    base_radius,
    density,
    youngs_modulus=E,
    shear_modulus=shear_modulus,
)

octopus_sim.append(octo_arm)

dt = 1e-4
octopus_sim.dampen(octo_arm).using(
    AnalyticalLinearDamper,
    damping_constant=nu,
    time_step=dt,
)

gravitational_acc = -9.80665
octopus_sim.add_forcing_to(octo_arm).using(
    GravityForces, acc_gravity=np.array([0.0, gravitational_acc, 0.0])
)
print("Gravity now acting on octopus arm")

period = 2.0
wave_length = 1.0
b_coeff = np.array([3.4e-3, 3.3e-3, 4.2e-3, 2.6e-3, 3.6e-3, 3.5e-3])

octopus_sim.add_forcing_to(octo_arm).using(
    MuscleTorques,
    base_length=base_length,
    b_coeff=b_coeff,
    period=period,
    wave_number=2.0 * np.pi / (wave_length),
    phase_shift=0.0,
    rest_lengths=octo_arm.rest_lengths,
    ramp_up_time=period,
    direction=normal,
    with_spline=True,
)
print("Muscle torques added to the octopus arm")

origin_plane = np.array([0.0, -base_radius, 0.0])
normal_plane = normal
slip_velocity_tol = 1e-8
froude = 0.1
mu = base_length / (period * period * np.abs(gravitational_acc) * froude)
kinetic_mu_array = np.array(
    [1.0 * mu, 1.5 * mu, 2.0 * mu]
)  # [forward, backward, sideways]
static_mu_array = 2 * kinetic_mu_array


octopus_sim.add_forcing_to(octo_arm).using(
    AnisotropicFrictionalPlane,
    k=1.0,
    nu=1e-6,
    plane_origin=origin_plane,
    plane_normal=normal_plane,
    slip_velocity_tol=slip_velocity_tol,
    static_mu_array=static_mu_array,
    kinetic_mu_array=kinetic_mu_array,
)
print("Friction forces added to the octopus arm")

start_force_x = 1.0 #change to whatever
start_force = np.array([start_force_x, 0.0, 0.0])
octopus_sim.add_forcing_to(octo_arm).using(
    EndpointForces, 
    start_force, 
    0.0*start_force, #this is the slot for end_force, and we want it to be 0, unless...?
    ramp_up_time = 1e-2
    )

class ContinuumOctoCallBack(CallBackBaseClass):
    """
    Call back function for continuum octopus arm
    """

    def __init__(self, step_skip: int, callback_params: dict):
        CallBackBaseClass.__init__(self)
        self.every = step_skip
        self.callback_params = callback_params

    def make_callback(self, system, time, current_step: int):

        if current_step % self.every == 0:

            self.callback_params["time"].append(time)
            self.callback_params["step"].append(current_step)
            self.callback_params["position"].append(system.position_collection.copy())
            self.callback_params["velocity"].append(system.velocity_collection.copy())
            self.callback_params["avg_velocity"].append(
                system.compute_velocity_center_of_mass()
            )

            self.callback_params["center_of_mass"].append(
                system.compute_position_center_of_mass()
            )
            self.callback_params["curvature"].append(system.kappa.copy())

            return


pp_list = defaultdict(list)
octopus_sim.collect_diagnostics(octo_arm).using(
    ContinuumOctoCallBack, step_skip=100, callback_params=pp_list
)
print("Callback function added to the simulator")

octopus_sim.finalize()

final_time = 5.0 * period
total_steps = int(final_time / dt)
print("Total steps", total_steps)

timestepper = PositionVerlet()

integrate(timestepper, octopus_sim, final_time, total_steps)


def plot_video_2D(plot_params: dict, video_name="video.mp4", margin=0.2, fps=15):
    !pip install matplotlib==3.7.5
    from matplotlib import pyplot as plt
    import matplotlib.animation as manimation

    t = np.array(plot_params["time"])
    positions_over_time = np.array(plot_params["position"])
    total_time = int(np.around(t[..., -1], 1))
    total_frames = fps * total_time
    step = round(len(t) / total_frames)

    print("creating video -- this can take a few minutes")
    FFMpegWriter = manimation.writers["ffmpeg"]
    metadata = dict(title="Movie Test", artist="Matplotlib", comment="Movie support!")
    writer = FFMpegWriter(fps=fps, metadata=metadata)

    fig = plt.figure()
    ax = fig.add_subplot(111)
    plt.axis("equal")
    rod_lines_2d = ax.plot(
        positions_over_time[0][2], positions_over_time[0][0], linewidth=3
    )[0]
    ax.set_xlim([0 - margin, 3 + margin])
    ax.set_ylim([-1.5 - margin, 1.5 + margin])
    with writer.saving(fig, video_name, dpi=100):
        with plt.style.context("default"):
            for i,time in enumerate(range(1, len(t), step)):
                rod_lines_2d.set_xdata(positions_over_time[time][2])
                rod_lines_2d.set_ydata(positions_over_time[time][0])

                writer.grab_frame()

    plt.close(fig)


filename_video = "continuum_snake.mp4"
plot_video_2D(pp_list, video_name=filename_video, margin=0.2, fps=125)

Video("continuum_snake.mp4")

imports done
Gravity now acting on octopus arm
Muscle torques added to the octopus arm
Friction forces added to the octopus arm
Callback function added to the simulator
Total steps 100000


100%|████████████████████████████████████████████████████████████████████████| 100000/100000 [00:13<00:00, 7510.44it/s]

Final time of simulation is :  9.999999999983364



ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\Swych\\anaconda3\\Lib\\site-packages\\matplotlib\\ft2font.cp311-win_amd64.pyd'
Consider using the `--user` option or check the permissions.



  Obtaining dependency information for matplotlib==3.7.5 from https://files.pythonhosted.org/packages/e3/b1/1b6c34b89173d6c206dc5a4028e8518b4dfee3569c13bdc0c88d0486cae7/matplotlib-3.7.5-cp311-cp311-win_amd64.whl.metadata
  Using cached matplotlib-3.7.5-cp311-cp311-win_amd64.whl.metadata (5.8 kB)
Using cached matplotlib-3.7.5-cp311-cp311-win_amd64.whl (7.5 MB)
creating video -- this can take a few minutes


In [6]:
#conda install ffmpeg
#!conda install conda=24.3.0

^C
